<a href="https://colab.research.google.com/github/iGhostlp/Proyecto_MELI/blob/master/Proyecto_MELI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prueba usando base de datos kaggle**

Librerias

In [ ]:
from pyspark import *
import pandas as pd
import numpy as np
from pyspark.sql import SQLContext
import pyspark.sql.functions as sqlf
import yaml
# import validations
#import matplotlib.pyplot as plt
from pyspark.sql.types import *
import pyarrow.parquet as pq
#Master
#from pyspark.sql import functions as F
#from pyspark.sql import SparkSession
pd.options.display.max_columns = 999
pd.set_option('max_colwidth',200)
#spark.conf.set("spark.sql.execution.arrow.enabled","false")
import kagglehub
from kagglehub import KaggleDatasetAdapter
import os

In [ ]:
# Descarga la última versión del dataset
dataset_path = kagglehub.dataset_download("davidcariboo/player-scores")

# Obtener todos los archivos CSV en el dataset
csv_files = [file for file in os.listdir(dataset_path) if file.endswith(".csv")]
print("Archivos encontrados:", csv_files)

# Diccionario para almacenar los DataFrames
df_dict = {}

# Leer y almacenar cada CSV en el diccionario
for file in csv_files:
    df_name = file.replace(".csv", "")  # Usar el nombre del archivo como clave
    df_dict[df_name] = pd.read_csv(os.path.join(dataset_path, file))

In [194]:
df_appearances = df_dict["appearances"]
df_club_games = df_dict["club_games"]
df_clubs = df_dict["clubs"]
df_competitions = df_dict["competitions"]
df_game_events = df_dict["game_events"]
df_games = df_dict["games"]
df_players = df_dict["players"]

In [195]:
df1 = df_competitions.filter(items=['competition_id','name','country_name','domestic_league_code','confederation'])

In [196]:
#Filtro las ligas principales
ligas_europa = ['GB1', 'FR1', 'IT1', 'ES1','L1']
df_liga_filtered = df1[df1['competition_id'].isin(ligas_europa)]

In [197]:
df_liga_filtered.head()

,competition_id,name,country_name,domestic_league_code,confederation
9,ES1,laliga,Spain,ES1,europa
10,FR1,ligue-1,France,FR1,europa
11,IT1,serie-a,Italy,IT1,europa
16,GB1,premier-league,England,GB1,europa
33,L1,bundesliga,Germany,L1,europa


In [212]:
# Renombro columna para posteriormente joinear
df_clubs = df_clubs.rename(columns={"domestic_competition_id":"competition_id"})

In [213]:
df_merged = df_liga_filtered.merge(df_clubs, on='competition_id', how='inner')

In [214]:
# Joineo los dataframes en base a la liga
df_merged.head()

,competition_id,name_x,country_name,domestic_league_code,confederation,club_id,club_code,name_y,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
0,ES1,laliga,Spain,ES1,europa,5358,sd-huesca,SD Huesca,NaN,24,27.5,8,33.3,2,El Alcoraz,9128,+€1.43m,NaN,2020,../data/raw/transfermarkt-scraper/2020/clubs.json.gz,https://www.transfermarkt.co.uk/sd-huesca/startseite/verein/5358
1,ES1,laliga,Spain,ES1,europa,993,fc-cordoba,Córdoba CF,NaN,24,27.9,4,16.7,0,Nuevo Arcángel,21822,+-0,NaN,2014,../data/raw/transfermarkt-scraper/2014/clubs.json.gz,https://www.transfermarkt.co.uk/fc-cordoba/startseite/verein/993
2,ES1,laliga,Spain,ES1,europa,1108,deportivo-alaves,Deportivo Alavés S.A.D.,NaN,25,26.5,10,40.0,4,Mendizorroza,19840,+€9.79m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/deportivo-alaves/startseite/verein/1108
3,ES1,laliga,Spain,ES1,europa,13,atletico-madrid,Club Atlético de Madrid S.A.D.,NaN,23,29.3,16,69.6,16,Riyadh Air Metropolitano,70460,€-91.10m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/atletico-madrid/startseite/verein/13
4,ES1,laliga,Spain,ES1,europa,131,fc-barcelona,Futbol Club Barcelona,NaN,26,25.0,8,30.8,14,Olímpic Lluís Companys,55926,+€6.40m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/fc-barcelona/startseite/verein/131


In [215]:
#Selecciono solo la temporada 2024
df_merged = df_merged[df_merged['last_season'] == 2024 ]

In [233]:
df_merged.head()

,competition_id,name_x,country_name,domestic_league_code,confederation,club_id,club_code,name_y,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
2,ES1,laliga,Spain,ES1,europa,1108,deportivo-alaves,Deportivo Alavés S.A.D.,NaN,25,26.5,10,40.0,4,Mendizorroza,19840,+€9.79m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/deportivo-alaves/startseite/verein/1108
3,ES1,laliga,Spain,ES1,europa,13,atletico-madrid,Club Atlético de Madrid S.A.D.,NaN,23,29.3,16,69.6,16,Riyadh Air Metropolitano,70460,€-91.10m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/atletico-madrid/startseite/verein/13
4,ES1,laliga,Spain,ES1,europa,131,fc-barcelona,Futbol Club Barcelona,NaN,26,25.0,8,30.8,14,Olímpic Lluís Companys,55926,+€6.40m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/fc-barcelona/startseite/verein/131
5,ES1,laliga,Spain,ES1,europa,366,real-valladolid,Real Valladolid Club de Fútbol S.A.D.,NaN,26,27.0,18,69.2,7,José Zorrilla,27618,+€975k,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/real-valladolid/startseite/verein/366
6,ES1,laliga,Spain,ES1,europa,1050,fc-villarreal,Villarreal Club de Fútbol S.A.D.,NaN,24,27.4,11,45.8,11,La Cerámica,23500,+€11.50m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/fc-villarreal/startseite/verein/1050


In [231]:
df_appearances = df_appearances[df_appearances['date'] >= '2024-01-01']

In [232]:
df_appearances.head()

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
1523086,4095269_105470,4095269,105470,31,31,2024-01-01,Alisson,GB1,0,0,0,0,90
1523087,4095269_134270,4095269,134270,762,762,2024-01-01,Dan Burn,GB1,0,0,0,0,82
1523088,4095269_135343,4095269,135343,762,762,2024-01-01,Fabian Schär,GB1,0,0,0,0,90
1523089,4095269_139208,4095269,139208,31,31,2024-01-01,Virgil van Dijk,GB1,0,0,0,0,90
1523090,4095269_146310,4095269,146310,31,31,2024-01-01,Wataru Endo,GB1,1,0,0,0,75


In [229]:
df_game_events = df_game_events[(df_game_events['date'] >= '2024-01-01') & (df_game_events['type'] == 'Goals')]

In [253]:
df_eventos =  df_merged.merge(df_game_events, on='club_id', how='inner')

In [ ]:
df_eventos.head()

In [255]:
#Sumamos los eventos de gol a fin de agrupar el equipo mas goleador
df_eventos['goals_count'] = df_eventos['type'].apply(lambda x: 1 if x == 'Goals' else 0)
goles_por_equipo = df_eventos.groupby("name_y")["goals_count"].sum().reset_index()
#goles_por_club.head()

In [257]:
#Por ultimo, ordenamos los clubes de acuerdo a quien marco mas goles
goles_por_equipo = goles_por_equipo.sort_values(by="goals_count", ascending=False)

In [258]:
goles_por_equipo.head(100)

,name_y,goals_count
37,Futbol Club Barcelona,203
67,Real Madrid Club de Fútbol,185
49,Liverpool Football Club,182
50,Manchester City Football Club,172
33,FC Bayern München,168
...,...,...
39,Fußball-Club St. Pauli von 1910,29
69,Real Valladolid Club de Fútbol S.A.D.,29
8,Association sportive de Saint-Étienne Loire,26
3,Angers Sporting Club de l'Ouest,26


In [192]:
# Paso 1: Selección de liga (Solo ligas principales de Europa)
ligas_europa = ["premier-league", "laliga", "bundesliga", "serie-a	", "ligue-1"]

print("\nElige una liga de Europa:")
for i, liga in enumerate(ligas_europa, 1):
    print(f"{i}. {liga}")

opcion = int(input("Ingresa el número de la liga: ").strip())
liga_elegida = ligas_europa[opcion - 1] if 1 <= opcion <= len(ligas_europa) else exit()

# Filtrar el dataset por la liga elegida
df_liga = df_liga_filtered[df_liga_filtered["name"] == liga_elegida]

# Agrupar por equipo y sumar los goles
goles_por_equipo = df_liga.groupby("home_club_name")["goals"].sum()

# Encontrar el equipo con más goles
equipo_sugerido = goles_por_equipo.idxmax() if not goles_por_equipo.empty else "No encontrado"

# Mostrar resultado
print(f"\n⚽ ¡El equipo con más goles en {liga_elegida} es {equipo_sugerido}! ⚽")


Elige una liga de Europa:
1. premier-league
2. laliga
3. bundesliga
4. serie-a	
5. ligue-1
Ingresa el número de la liga: 1


KeyError: 'home_club_name'

In [ ]:
import pandas as pd

# Cargar el dataset de Kaggle "player-scores"
df = pd.read_csv("player_scores.csv")

# Paso 1: Selección de liga (Solo ligas principales de Europa)
ligas_europa = ["Premier League", "La Liga", "Bundesliga", "Serie A", "Ligue 1"]

print("\nElige una liga de Europa:")
for i, liga in enumerate(ligas_europa, 1):
    print(f"{i}. {liga}")

opcion = int(input("Ingresa el número de la liga: ").strip())
liga_elegida = ligas_europa[opcion - 1] if 1 <= opcion <= len(ligas_europa) else exit()

# Filtrar el dataset por la liga elegida
df_liga = df[df["league"] == liga_elegida]

# Agrupar por equipo y sumar los goles
goles_por_equipo = df_liga.groupby("team")["goals"].sum()

# Encontrar el equipo con más goles
equipo_sugerido = goles_por_equipo.idxmax() if not goles_por_equipo.empty else "No encontrado"

# Mostrar resultado
print(f"\n⚽ ¡El equipo con más goles en {liga_elegida} es {equipo_sugerido}! ⚽")